In [1]:
import requests
from bs4 import BeautifulSoup


def crawler_stats(player):
    aux = player.name_url.split('_')
    url = "https://www.espn.com/nba/player/stats/_" + aux[1]
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    
    tables = soup.find_all('table')
    
    datasets = []
    for table in tables:
        header_row = table.select('thead tr')[0]
        columns = [th.text.strip() for th in header_row.select('th')]
        body_rows = table.select('tbody tr')
        values = []
        for row in body_rows:
            row_data = [td.text.strip() for td in row.select('td')]
            value = dict(zip(columns, row_data))
            values.append({'url': url, **value})

        datasets.append(values)

    return datasets

In [8]:
import pandas as pd

df = pd.read_csv('../dataset/scraping/players.csv')

stats = []
for i, value in df.iterrows():
    stat = crawler_stats(value)
    stats.append(stat)

In [33]:
averages = []
totals = []
miscs = []
fails = []
for index, stat in enumerate(stats):
    try:
        for i in range(0, len(stat[0])):
            averages.append({**stat[0][i], **stat[1][i]})

        for i in range(0, len(stat[2])):
            totals.append({**stat[2][i], **stat[3][i]})

        for i in range(0, len(stat[4])):
            miscs.append({**stat[4][i], **stat[5][i]})
    except:
        fails.append(stat)

In [42]:
df1 = pd.DataFrame(data=averages)

df1.to_csv('../dataset/scraping/averages.csv', index=False)

In [43]:
df2 = pd.DataFrame(data=totals)

df2.to_csv('../dataset/scraping/totals.csv', index=False)

In [44]:
df3 = pd.DataFrame(data=miscs)

df3.to_csv('../dataset/scraping/miscs.csv', index=False)